In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
### 보고 싶은 날짜 입력(주 단위로 업데이트)
# - s_date : 시작일  - e_date : 종료일 
s_date = 20200101
e_date = 20221231

### 공통 api key
apikey = "qPwOeIrU-2306-FHRACT-0598"

### open api_1 해파리 정보 for 보고서 날짜 확인
# - open api_1 호출
url = "https://www.nifs.go.kr/OpenAPI_xml?id=jellyList&key={key}&sdate={sd}&edate={ed}"
api_url = url.format(key = apikey, sd = s_date, ed = e_date)

rs_data = requests.get(api_url)
rs_text = rs_data.text
rs_xml = BeautifulSoup(rs_text, "lxml")

# - 보고서 게시일 리스트 생성
list_date = []
for val in rs_xml.find_all("board_idx") :
    if len(val.text) == 10 :
        list_date.append(val.text)

        
### open api_3 해파리 상세정보 - 속보 for 실제 사용할 데이터
# - open api_3 호출
url = "https://www.nifs.go.kr/OpenAPI_xml?id=jellyDetail2&key={key}&srcode={srcode}"  

## 설정한 날짜로 통합 데이터프레임 생성
# - 데이터 : "조사일시", "해파리종류", "독성", "발견율(%)", "출현밀집구분", "출현해역"
# - 관측 날짜 내림차순
df_jf = pd.DataFrame()
sido_list = ["부산","인천","울산","강원","충남","전북","전남","경북","경남","제주"]

# - 대제목, 중제목 구분자 함수 호출
def getOpt_Dict(num) :
    temp_dict = {1:"◎", 2:"○"}
    if num <= 10 :
        temp_dict = {1:"○", 2:"#8226;"}
    return temp_dict 

# - 조사일별 데이터 불러들이기
for val in list_date :    
    api_url = url.format(key = apikey, srcode = val)
    rs_data = requests.get(api_url)
    rs_text = rs_data.text
    rs_xml = BeautifulSoup(rs_text, "lxml")
    
    # - 임시 데이터프레임 생성 for 추가 데이터
    temp_df = pd.DataFrame(columns=["조사일시", "해파리종류", "독성", "발견율(%)", "출현밀집구분", "출현해역"])

    # - 같은 조사일 다른 해파리, 데이터 프레임 생성
    for i in range(len(rs_xml.find_all("jf_name"))) :
        temp_df["조사일시"] = [rs_xml.find("rdate").text]
        temp_df["해파리종류"] = [rs_xml.find_all("jf_name")[i].text]
        temp_df["독성"] = [rs_xml.find_all("jf_kbn")[i].text]
        temp_df["발견율(%)"] = [re.findall(r'\d+\.\d+', rs_xml.find_all("week_ratio_desc")[i].text)[0]]
        
        # - xml 텍스트 추출 (대제목, 소제목으로 구분하여 데이터 분리)
        opt_dict = getOpt_Dict(str(rs_xml.find_all("news_desc")[i]).replace("<news_desc>", 
                                                                    "").replace("</news_desc>", "").find("○"))
        split_data = str(rs_xml.find_all("news_desc")[i]).replace("<news_desc>", 
                                                          "").replace("</news_desc>", "").split(opt_dict[2])[1:]
        
        
        # - 중제목 데이터 -> "출현밀집구분", 중제목 하위 데이터 -> "출현해역"
        for j in range(len(split_data)) :
            temp_df["출현밀집구분"] = [split_data[j][:3]]
            
            # - "-" 구분으로 행 분리
            temp_1 = split_data[j].split("- ")[1:]
            temp_list = [val.split("\r")[0].rstrip() for val in temp_1]
            
            # - "," 지역 분리된 값 통합
            temp_new = []

            for k in range(len(temp_list)):
                if temp_list[k][-1] == "," :
                    temp_new.append(" ".join(temp_list[k:(k+2)]))
                elif (temp_list[k][:2] in sido_list)&(temp_list[k][-1] != ",") :
                    temp_new.append(temp_list[k])
                else :
                    continue
                    
            # - 지역 중복값 제거
            temp_unique = list(set(temp_new))
            
            # - 출현해역 :
            for v in temp_unique :
                temp_df["출현해역"] = [v]
                df_jf= pd.concat([df_jf, temp_df], ignore_index=True)

                    
### 데이터프레임 타입 변경
# - 조사일시 : datetime
# - 발견율(%) : float
df_jf["조사일시"] = pd.to_datetime(df_jf["조사일시"].str.slice(0, 10))
df_jf["발견율(%)"] = df_jf["발견율(%)"].astype(float)

### "독성" 컬럼의 결측치 -> "확인요망"으로 변경
for idx in range(len(df_jf)) :
    if df_jf["독성"][idx] == "" :
       df_jf["독성"][idx] = "확인요망"

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\USER\anaconda3\envs\busan_env\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"`

C:\Users\USER\AppData\Local\Temp\ipykernel_8756\1236510533.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jf["독성"][idx] = "확인요망"
C:\Users\USER\AppData\Local\Temp\ipykernel_8756\1236510533.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jf["독성"][idx] = "확인요망"
C:\Users\USER\AppData\Local\Temp\ipykernel_8756\1236510533.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jf["독성"][idx] = "확인요망"
C:\Users\USER\AppData\

In [3]:
df_jf

,조사일시,해파리종류,독성,발견율(%),출현밀집구분,출현해역
0,2022-12-02,살파류(척삭동물),무해성,0.50,저밀도,경남 남해 앵강만 연안
1,2022-12-02,보름달물해파리,약독성,0.50,저밀도,경북 영덕 구계항 근해
2,2022-12-02,노무라입깃해파리,강독성,2.70,저밀도,강원 속초 장사항 연안
3,2022-12-02,노무라입깃해파리,강독성,2.70,저밀도,부산 가덕도 연안
4,2022-12-02,노무라입깃해파리,강독성,2.70,저밀도,경북 울릉도-강릉 사이
...,...,...,...,...,...,...
5259,2020-05-01,노무라입깃해파리,강독성,0.48,저밀도,경남 고성 동해면 부근
5260,2020-05-01,두빛보름달해파리,강독성,1.92,저밀도,"강원 양양 동산항, 남애항 연안"
5261,2020-05-01,두빛보름달해파리,강독성,1.92,저밀도,강원 속초 용촌리 연안
5262,2020-05-01,두빛보름달해파리,강독성,1.92,저밀도,강원 동해 묵호항 근해


In [4]:
df_jf.to_excel("./해파리total.xlsx")